In [1]:
import csv
import numpy as np

''' Data loading ''' 

DATAPATH_IDEAL = 'ideal.csv'
DATAPATH_RES0 = 'residual_1_51.csv'
DATAPATH_RES1 = 'residual_52_54.csv'
DATAPATH_RES2 = 'residual_55_56.csv'

with open(DATAPATH_IDEAL, newline='') as csvfile:
  reader = csv.DictReader(csvfile, delimiter='\t')
  ideal_arraydict = [row for row in reader]

with open(DATAPATH_RES0, newline='') as csvfile:
  reader = csv.DictReader(csvfile, delimiter='\t')
  residual0_arraydict = [row for row in reader]

with open(DATAPATH_RES1, newline='') as csvfile:
  reader = csv.DictReader(csvfile, delimiter='\t')
  residual1_arraydict = [row for row in reader]

with open(DATAPATH_RES2, newline='') as csvfile:
  reader = csv.DictReader(csvfile, delimiter='\t')
  residual2_arraydict = [row for row in reader]

''' Data prep '''
n_ideal = np.array([float(obj["n_i"]) for obj in ideal_arraydict])
g_ideal = np.array([float(obj["g_i"]) for obj in ideal_arraydict])

# Compile shared residual coefficients
n_res = np.hstack(([float(obj["n_i"]) for obj in residual0_arraydict],
  [float(obj["n_i"]) for obj in residual1_arraydict],
  [float(obj["n_i"]) for obj in residual2_arraydict]))
d_res = np.hstack(([float(obj["d_i"]) for obj in residual0_arraydict],
  [float(obj["d_i"]) for obj in residual1_arraydict],
  [1.0 for obj in residual2_arraydict]))
t_res = np.hstack(([float(obj["t_i"]) for obj in residual0_arraydict],
  [float(obj["t_i"]) for obj in residual1_arraydict],
  [0.0 for obj in residual2_arraydict]))

c_res1_51 = np.array([float(obj["c_i"]) for obj in residual0_arraydict])

c_res52_54 = np.array([float(obj["c_i"]) for obj in residual1_arraydict])
alpha_res52_54 = np.array([float(obj["alpha_i"]) for obj in residual1_arraydict])
beta_res52_54 = np.array([float(obj["beta_i"]) for obj in residual1_arraydict])
gamma_res52_54 = np.array([float(obj["gamma_i"]) for obj in residual1_arraydict])
eps_res52_54 = np.array([float(obj["eps_i"]) for obj in residual1_arraydict])

a_res55_56 = np.array([float(obj["a_i"]) for obj in residual2_arraydict])
b_res55_56 = np.array([float(obj["b_i"]) for obj in residual2_arraydict])
B_res55_56 = np.array([float(obj["B_i"]) for obj in residual2_arraydict])
C_res55_56 = np.array([float(obj["C_i"]) for obj in residual2_arraydict])
D_res55_56 = np.array([float(obj["D_i"]) for obj in residual2_arraydict])
A_res55_56 = np.array([float(obj["A_i"]) for obj in residual2_arraydict])
beta_res55_56 = np.array([float(obj["beta_i"]) for obj in residual2_arraydict])

''' Set up static parameters '''
Tc = 647.096  # K
rhoc = 322    # kg / m^3
R = 0.46151805 # kJ / kg K
# Generic precomputation
_exp1_55_56 = 0.5 / (beta_res55_56)

In [2]:
''' Simulate input '''
rho = 838.025
T = 500

rho = 358
T = 647

In [3]:
''' Compute reduced variables '''

# Reciprocal reduced volume
d = rho / rhoc
# Reciprocal reduced temperature
t = Tc / T

In [4]:
def phi(d, t):
  ''' Reduced Helmholtz function f/(RT). ''' 

  # Scalar precomputation
  d_quad = (d-1.0)**2

  ''' Ideal-gas part of Helmholtz function f '''
  phi0 = np.log(d) + n_ideal[0] + n_ideal[1] * t + n_ideal[2] * np.log(t)
  phi0 += np.dot(n_ideal[3:8], np.log(1.0 - np.exp(-g_ideal[3:8] * t)))

  ''' Residual part of Helmholtz function f
  Evaluated using two registers that combine as
    np.dot( coeffs, np.exp(exponents) ).
  '''
  # Allocate exponent cache
  exponents = np.zeros_like(n_res)
  # Allocate and evaluate coeffs
  coeffs = n_res * (d ** d_res) * (t ** t_res)

  # Compute distance term for 1-indices 55 to 56
  theta = (1 - t) + A_res55_56 * d_quad ** _exp1_55_56
  Delta = theta**2 + B_res55_56 * d_quad ** a_res55_56
  # Factor in Delta**b_i term for 1-indices from 55 to 56
  coeffs[54:56] *= Delta ** b_res55_56

  # Compute exponents for 1-indices 8 to 51 as -d**c_i
  exponents[7:51] = -d ** c_res1_51[7:51]
  # Compute exponents for 1-indices from 52 to 54
  exponents[51:54] = -alpha_res52_54 * (d - eps_res52_54) ** 2 \
    -beta_res52_54*(t - gamma_res52_54)**2
  # Compute exponents for 1-indices from 55 to 56
  exponents[54:56] = -C_res55_56 * d_quad \
    -D_res55_56*(t - 1)**2

  # Compute residual part of reduced Helmholtz
  phir = np.dot(coeffs, np.exp(exponents))

  return (phi0, phir)

def Z(d, t):
  ''' Compressibility factor evaluation
  The compressibility factor Z is given by
    Z == p * v / (R * T) = 1 + delta * d(phi_R)/d(delta),
  where d(phi_R)/d(delta) is the partial derivative of the residual
  part of the reduced Helmholtz function, with respect to d = rho/rhoc.
  Evaluated using two registers that combine as
    np.dot( coeffs, np.exp(exponents) ).
  '''

  # Scalar precomputation
  d_quad = (d-1)**2

  # Allocate exponent cache
  exponents = np.zeros_like(n_res)
  # Allocate and partially evaluate coeffs
  coeffs = n_res * (d ** (d_res-1)) * (t ** t_res)
  # Factor in d_i - c_i * d**c_i term
  coeffs[0:51] *= (d_res[0:51] - c_res1_51 * d ** c_res1_51)
  coeffs[51:54] *= d_res[51:54] - 2.0 * alpha_res52_54 * d * (d - eps_res52_54)

  # Compute distance term for 1-indices 55 to 56
  theta = (1 - t) + A_res55_56 * d_quad ** _exp1_55_56
  Delta = theta**2 + B_res55_56 * d_quad ** a_res55_56
  # Factor in other terms for 1-indices from 55 to 56 in two steps
  coeffs[54:56] *= (
    Delta * (1.0 - 2.0 * C_res55_56 * (d-1.0) * d)
    + b_res55_56 * d * (d-1.0) * (
      A_res55_56 * theta * 2 / beta_res55_56 * d_quad**(_exp1_55_56 - 1.0)
      + 2 * B_res55_56 * a_res55_56 * d_quad**(a_res55_56 - 1.0)
    )
  )
  coeffs[54:56] *= Delta ** np.where(Delta != 0, b_res55_56-1, 1.0)

  # Compute exponents for 1-indices 8 to 51 as -d**c_i
  exponents[7:51] = -d ** c_res1_51[7:51]
  # Compute exponents for 1-indices from 52 to 54
  exponents[51:54] = -alpha_res52_54 * (d - eps_res52_54) ** 2 \
    -beta_res52_54*(t - gamma_res52_54)**2
  # Compute exponents for 1-indices from 55 to 56
  exponents[54:56] = -C_res55_56 * d_quad \
    -D_res55_56*(t - 1)**2

  # Reduce
  phir_d = np.dot(coeffs, np.exp(exponents))
  return 1 + d * phir_d


In [5]:
''' Compute reduced variables '''
# Reciprocal reduced volume
d = rho / rhoc
# Reciprocal reduced temperature
t = Tc / T

# phi, phir_d, Z
print(f"phi0 = {phi(d, t)[0]:.8f}, phir = {phi(d, t)[1]:.8f}, phir_d = {(Z(d, t) - 1)/d:.9f}")

# Print verification values
verification_string = '''
For T = 647 K, rho = 358 kg m^{-3}:
phi0 = -1.56319605
phir = -1.21202657
phir_d = -0.714012024
'''
print(verification_string)


phi0 = -1.56319605, phir = -1.21202657, phir_d = -0.714012024

For T = 647 K, rho = 358 kg m^{-3}:
phi0 = -1.56319605
phir = -1.21202657
phir_d = -0.714012024



In [6]:
from time import perf_counter
# Timing for pressure evaluation
N_timing = 1e4
t1 = perf_counter()
for i in range(N_timing):
  p = Z(rho / rhoc, Tc / T) * rho * R * T
t2 = perf_counter()

t1_ideal = perf_counter()
for i in range(N_timing):
  p_ideal = rho * R * T
t2_ideal = perf_counter()

###  Free evaluation

In [7]:
''' Helmholtz evaluation ''' 

''' Ideal-gas part of Helmholtz function f '''
phi0 = np.log(d) + n_ideal[0] + n_ideal[1] * t + n_ideal[2] * np.log(t)
phi0 += np.dot(n_ideal[3:8], np.log(1.0 - np.exp(-g_ideal[3:8] * t)))

''' Residual part of Helmholtz function f
Evaluated using two registers that combine as
  np.dot( coeffs, np.exp(exponents) ).
'''

# Scalar precomputation
d_quad = (d-1)**2

# Allocate exponent cache
exponents = np.zeros_like(n_res)
# Allocate and evaluate coeffs
coeffs = n_res * (d ** d_res) * (t ** t_res)

# Compute distance term for 1-indices 55 to 56
theta = (1 - t) + A_res55_56 * d_quad ** _exp1_55_56
Delta = theta**2 + B_res55_56 * d_quad ** a_res55_56

# Factor in Delta**b_i term for 1-indices from 55 to 56
coeffs[54:56] *= Delta ** b_res55_56

# Compute exponents for 1-indices 8 to 51 as -d**c_i
exponents[7:51] = -d ** c_res1_51[7:51]
# Compute exponents for 1-indices from 52 to 54
exponents[51:54] = -alpha_res52_54 * (d - eps_res52_54) ** 2 \
  -beta_res52_54*(t - gamma_res52_54)**2
# Compute exponents for 1-indices from 55 to 56
exponents[54:56] = -C_res55_56 * d_quad \
  -D_res55_56*(t - 1)**2

# Add residual part
phir = np.dot(coeffs, np.exp(exponents))

In [8]:
''' Compressibility factor evaluation
The compressibility factor Z is given by
  Z == p * v / (R * T) = 1 + delta * d(phi_R)/d(delta).
''' 

''' Partial derivative of residual part of Helmholtz function.
Evaluated using two registers that combine as
  np.dot( coeffs, np.exp(exponents) ).
'''

# Scalar precomputation
d_quad = (d-1)**2

# Allocate exponent cache
exponents = np.zeros_like(n_res)
# Allocate and partially evaluate coeffs
coeffs = n_res * (d ** (d_res-1)) * (t ** t_res)

# Compute distance term for 1-indices 55 to 56
theta = (1 - t) + A_res55_56 * d_quad ** _exp1_55_56
Delta = theta**2 + B_res55_56 * d_quad ** a_res55_56

# Factor in d_i - c_i * d**c_i term
coeffs[0:51] *= (d_res[0:51] - c_res1_51 * d ** c_res1_51)
coeffs[51:54] *= d_res[51:54] - 2.0 * alpha_res52_54 * d * (d - eps_res52_54)
# Factor in other terms for 1-indices from 55 to 56 in two steps
coeffs[54:56] *= (
  Delta * (1.0 - 2.0 * C_res55_56 * (d-1.0) * d)
  + b_res55_56 * d * (d-1.0) * (
    A_res55_56 * theta * 2 / beta_res55_56 * d_quad**(_exp1_55_56 - 1.0)
    + 2 * B_res55_56 * a_res55_56 * d_quad**(a_res55_56 - 1.0)
  )
)
coeffs[54:56] *= Delta ** np.where(Delta != 0, b_res55_56-1, 1.0)

# Compute exponents for 1-indices 8 to 51 as -d**c_i
exponents[7:51] = -d ** c_res1_51[7:51]
# Compute exponents for 1-indices from 52 to 54
exponents[51:54] = -alpha_res52_54 * (d - eps_res52_54) ** 2 \
  -beta_res52_54*(t - gamma_res52_54)**2
# Compute exponents for 1-indices from 55 to 56
exponents[54:56] = -C_res55_56 * d_quad \
  -D_res55_56*(t - 1)**2

# Reduce
phir_d = np.dot(coeffs, np.exp(exponents))
Z = 1 + d * phir_d